# Spark OCR in Healthcare

Spark OCR is a commercial extension of Spark NLP for optical character recognition from images, scanned PDF documents, Microsoft DOCX and DICOM files. 

In this notebook we will:
  - Import clinical notes in pdf format and store in delta
  - Convert pdfs to image and improve image quality
  - Extract text from pdfs and store resulting text data in delta 

In [10]:
%pip install transformers==4.22.1 johnsnowlabs spark-nlp 

StatementMeta(, 6456efdc-8cfc-46b1-b9e1-6361ebfe6479, -1, Finished, Available)


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /nfs4/pyenv-dbbbce56-9d07-4623-a3a1-862df04c3c80/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# To prevent undesired infor from the outputs
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)

StatementMeta(, 6456efdc-8cfc-46b1-b9e1-6361ebfe6479, 10, Finished, Available)

In [11]:
import os
import json
import string
#import sys
#import base64
import numpy as np
import pandas as pd
 
import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader
 
import sparkocr
from sparkocr.transformers import *
from sparkocr.utils import *
from sparkocr.enums import *
 
from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel
from sparknlp.training import CoNLL
 
import matplotlib.pyplot as plt
 
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)
 
spark.sql("set spark.sql.legacy.allowUntypedScalaUDF=true")
 
print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())
print('sparkocr : ',sparkocr.version())

StatementMeta(, 6456efdc-8cfc-46b1-b9e1-6361ebfe6479, 33, Finished, Available)

ModuleNotFoundError: No module named 'sparknlp_jsl'